In [1]:
import cv2
import matplotlib.pyplot as plt
import glob
import numpy as np

In [37]:
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D
from keras.models import Model
from keras import backend as K

input_img = Input(shape=(28, 28, 1))  # adapt this if using `channels_first` image data format

x = Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

# at this point the representation is (4, 4, 8) i.e. 128-dimensional

x = Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(16, (3, 3), activation='relu')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

In [38]:
all_images = []
img_list = glob.glob('resized_images/*.jpg')

In [39]:
for image in img_list:
    img = cv2.imread(image,0)
#     img = img.astype('float32') / 255.
    img = np.reshape(img,(28,28,1))
    all_images.append(img)
all_images = np.array(all_images)

In [40]:
all_images.shape

(1032, 28, 28, 1)

In [41]:
train = all_images[:700]

In [42]:
test = all_images[700:]

In [43]:
autoencoder.fit(train, train,
                epochs=100,
                batch_size=128,
                shuffle=True,
                validation_data=(test, test))

Train on 700 samples, validate on 332 samples
Epoch 1/100
700/700 [==============================] - 4s 5ms/step - loss: -1200.1935 - val_loss: -1786.2943
Epoch 2/100
700/700 [==============================] - 2s 3ms/step - loss: -1802.3787 - val_loss: -1794.3291
Epoch 3/100
700/700 [==============================] - 2s 3ms/step - loss: -1806.6238 - val_loss: -1795.7928
Epoch 4/100
700/700 [==============================] - 2s 3ms/step - loss: -1807.6330 - val_loss: -1796.3157
Epoch 5/100
700/700 [==============================] - 2s 3ms/step - loss: -1808.0364 - val_loss: -1796.5756
Epoch 6/100
700/700 [==============================] - 2s 3ms/step - loss: -1808.2460 - val_loss: -1796.7205
Epoch 7/100
700/700 [==============================] - 2s 3ms/step - loss: -1808.3758 - val_loss: -1796.8078
Epoch 8/100
700/700 [==============================] - 2s 3ms/step - loss: -1808.4563 - val_loss: -1796.8678
Epoch 9/100
700/700 [==============================] - 2s 3ms/step - loss: -1808.5

In [10]:
autoencoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 16)        160       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 8)         1160      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 7, 7, 8)           0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 7, 7, 8)           584       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 4, 4, 8)           0         
__________

In [44]:
autoencoder.save('model2.h5')

In [45]:
predicted = autoencoder.predict([test[1:2]])

In [46]:
predicted.shape

(1, 28, 28, 1)

In [19]:
from scipy import spatial

In [48]:
actual = test[1:2]

In [49]:
flat1 = predicted.flatten()
flat2 = actual.flatten()

In [50]:
result = 1 - spatial.distance.cosine(flat1,flat2)
result

11.132881281511061

In [51]:
cat_img = cv2.imread('cat.jpg',0)
cat_img = cv2.resize(cat_img,(28,28))
# cat_img = cat_img.astype('float32') / 255.
cat_img = np.reshape(cat_img,(1,28,28,1))

In [52]:
cat_prediction = autoencoder.predict(cat_img)

In [53]:
cat_prediction.shape

(1, 28, 28, 1)

In [54]:
cat_pred_flat = cat_prediction.flatten()
cat_flat = cat_img.flatten()

In [55]:
result = 1 - spatial.distance.cosine(cat_flat,cat_pred_flat)
result

14.282374352996786